In [1]:
"""
Signalmonitor für Raspberry Pi mit MCC DAQ HAT
Liest analoge Signale und stellt sie graphisch dar.
"""

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import ipywidgets as widgets
from IPython.display import display, clear_output
import time
from datetime import datetime
import pandas as pd
from scipy import signal as sig

# Abtastrate und andere Konstanten
ABTASTRATE = 10000  # Hz
MAX_SPANNUNG = 5.0  # Maximale Eingangsspannung in Volt
PUFFER_GROESSE = 1000  # Anzahl der Datenpunkte im Puffer

class Signalleser:
    """
    Klasse zum Lesen von analogen Signalen vom ADC.
    """
    
    def __init__(self, abtastrate=ABTASTRATE, kanal=0):
        """
        Initialisiert den Signalleser.
        
        Args:
            abtastrate (int): Abtastrate in Hz
            kanal (int): ADC-Kanal
        """
        self.abtastrate = abtastrate
        self.kanal = kanal
        self.puffer = np.zeros(PUFFER_GROESSE)
        self.puffer_index = 0
        
    def lese_wert(self):
        """
        Liest einen Wert vom ADC.
        
        Returns:
            float: Gemessene Spannung in Volt
        
        Hinweis: Diese Funktion muss an die tatsächliche Hardware angepasst werden.
        """
        # Hier den Code zum Lesen vom ADC einfügen
        # Beispiel für MCC DAQ (muss angepasst werden):
        """
        from mcculw import ul
        from mcculw.enums import ULRange
        
        # Lese Wert vom ADC (Gerät, Kanal, Bereich)
        wert = ul.a_in(0, self.kanal, ULRange.BIP5VOLTS)
        spannung = ul.to_eng_units(0, ULRange.BIP5VOLTS, wert)
        return spannung
        """
        # Simulierte Implementierung (generiert ein synthetisches Signal):
        t = time.time()
        # Einfaches Sinus-Signal mit Rauschen
        spannung = 2.5 * np.sin(2 * np.pi * 10 * t) + 0.1 * np.random.randn()
        # Begrenzen der Spannung auf MAX_SPANNUNG
        return max(min(spannung, MAX_SPANNUNG), -MAX_SPANNUNG)
    
    def lese_block(self, anzahl_punkte):
        """
        Liest einen Block von Werten vom ADC.
        
        Args:
            anzahl_punkte (int): Anzahl der zu lesenden Datenpunkte
            
        Returns:
            tuple: (zeit, spannungen)
        """
        spannungen = np.zeros(anzahl_punkte)
        
        # Zeitmessung für die Abtastrate starten
        start_zeit = time.time()
        
        for i in range(anzahl_punkte):
            spannungen[i] = self.lese_wert()
            
            # Echtzeit-Abtastung versuchen
            soll_zeit = start_zeit + i / self.abtastrate
            jetzt = time.time()
            if jetzt < soll_zeit:
                time.sleep(soll_zeit - jetzt)
        
        ende_zeit = time.time()
        tatsaechliche_abtastrate = anzahl_punkte / (ende_zeit - start_zeit)
        
        zeit = np.linspace(0, (anzahl_punkte - 1) / self.abtastrate, anzahl_punkte)
        return zeit, spannungen, tatsaechliche_abtastrate
    
    def puffer_aktualisieren(self):
        """
        Aktualisiert den Signalpuffer mit einem neuen Wert.
        
        Returns:
            np.array: Aktueller Pufferinhalt
        """
        self.puffer[self.puffer_index] = self.lese_wert()
        self.puffer_index = (self.puffer_index + 1) % PUFFER_GROESSE
        
        # Sortieren des Puffers, so dass der älteste Wert zuerst kommt
        sortierter_puffer = np.roll(self.puffer, -self.puffer_index)
        return sortierter_puffer


class Signalanalyse:
    """
    Klasse für die Analyse von Signalen.
    """
    
    @staticmethod
    def berechne_frequenz(signal_daten, abtastrate):
        """
        Berechnet die Hauptfrequenz eines Signals mittels FFT.
        
        Args:
            signal_daten (np.array): Signaldaten
            abtastrate (float): Abtastrate in Hz
            
        Returns:
            float: Frequenz in Hz
        """
        # Entfernen von DC-Offset
        signal_daten = signal_daten - np.mean(signal_daten)
        
        # FFT berechnen
        n = len(signal_daten)
        yf = np.abs(np.fft.rfft(signal_daten))
        xf = np.fft.rfftfreq(n, 1 / abtastrate)
        
        # Maximale Frequenz finden (ignoriert DC)
        if len(xf) > 1:
            max_index = np.argmax(yf[1:]) + 1  # DC-Komponente ignorieren
            return xf[max_index]
        return 0
    
    @staticmethod
    def berechne_amplitude(signal_daten):
        """
        Berechnet die Amplitude eines Signals (Peak-to-Peak).
        
        Args:
            signal_daten (np.array): Signaldaten
            
        Returns:
            float: Amplitude in V
        """
        return np.max(signal_daten) - np.min(signal_daten)
    
    @staticmethod
    def berechne_mittelwert(signal_daten):
        """
        Berechnet den Mittelwert eines Signals.
        
        Args:
            signal_daten (np.array): Signaldaten
            
        Returns:
            float: Mittelwert in V
        """
        return np.mean(signal_daten)
    
    @staticmethod
    def berechne_effektivwert(signal_daten):
        """
        Berechnet den Effektivwert (RMS) eines Signals.
        
        Args:
            signal_daten (np.array): Signaldaten
            
        Returns:
            float: RMS in V
        """
        # DC-Offset entfernen für AC-RMS
        signal_ac = signal_daten - np.mean(signal_daten)
        return np.sqrt(np.mean(np.square(signal_ac)))
    
    @staticmethod
    def erkenne_signaltyp(signal_daten, abtastrate):
        """
        Versucht, den Signaltyp zu erkennen.
        
        Args:
            signal_daten (np.array): Signaldaten
            abtastrate (float): Abtastrate in Hz
            
        Returns:
            str: Erkannter Signaltyp
        """
        # Einfache Heuristik zur Signalerkennung
        
        # DC-Offset entfernen
        signal_norm = signal_daten - np.mean(signal_daten)
        
        # Krestwert-Faktor (Crest factor)
        peak = np.max(np.abs(signal_norm))
        rms = np.sqrt(np.mean(np.square(signal_norm)))
        if rms > 0:
            crest_factor = peak / rms
        else:
            crest_factor = 0
        
        # Verschiedene Signaltypen haben unterschiedliche Krestwerte
        if crest_factor < 1.2:  # Fast konstant
            return "Gleichspannung (DC)"
        elif 1.3 < crest_factor < 1.5:  # ~ 1.414 für Sinuswelle
            return "Sinuswelle"
        elif 1.6 < crest_factor < 2.0:  # ~ 1.73 für Dreieckwelle
            return "Dreieckwelle"
        elif crest_factor > 2.8:  # Höher für Rechteck, Puls, etc.
            # Weitere Analyse nach Zero-Crossings
            zero_cross = np.where(np.diff(np.signbit(signal_norm)))[0]
            if len(zero_cross) > 1:
                intervals = np.diff(zero_cross)
                if np.std(intervals) / np.mean(intervals) < 0.1:
                    return "Rechteckwelle"
                else:
                    return "Sägezahnwelle"
        
        return "Komplexes Signal"


class SignalmonitorGUI:
    """
    GUI für den Signalmonitor.
    """
    
    def __init__(self):
        """
        Initialisiert die GUI.
        """
        self.signalleser = Signalleser()
        self.analyse = Signalanalyse()
        self.laufend = False
        self.aufzeichnung_aktiv = False
        self.gemessene_daten = []
        self.aufzeichnungs_start = None
        self.animation = None
        self.setup_gui()
        
    def setup_gui(self):
        """
        Erstellt die GUI-Elemente.
        """
        # Betriebsmodus-Auswahl
        self.modus_tabs = widgets.Tab()
        
        # Echtzeit-Tab
        self.echtzeit_tab = widgets.VBox()
        
        # Einzelmessung-Tab
        self.einzelmessung_tab = widgets.VBox()
        
        # FFT-Tab
        self.fft_tab = widgets.VBox()
        
        # Tabs zusammenfügen
        self.modus_tabs.children = [self.echtzeit_tab, self.einzelmessung_tab, self.fft_tab]
        self.modus_tabs.set_title(0, "Echtzeit")
        self.modus_tabs.set_title(1, "Einzelmessung")
        self.modus_tabs.set_title(2, "FFT-Analyse")
        
        # Status-Anzeige
        self.status_text = widgets.HTML(value="<b>Status:</b> Bereit")
        
        # Messparameter
        # Abtastrate
        self.abtastrate_slider = widgets.IntSlider(
            value=10000,
            min=1000,
            max=100000,
            step=1000,
            description='Abtastrate (Hz):',
            style={'description_width': 'initial'}
        )
        
        # Anzeigedauer
        self.zeitbasis_slider = widgets.FloatSlider(
            value=0.1,
            min=0.01,
            max=1.0,
            step=0.01,
            description='Zeitbasis (s):',
            style={'description_width': 'initial'}
        )
        
        # Trigger-Einstellungen
        self.trigger_checkbox = widgets.Checkbox(
            value=False,
            description='Trigger aktivieren',
            style={'description_width': 'initial'}
        )
        
        self.trigger_level_slider = widgets.FloatSlider(
            value=0.0,
            min=-MAX_SPANNUNG,
            max=MAX_SPANNUNG,
            step=0.1,
            description='Trigger-Level (V):',
            style={'description_width': 'initial'}
        )
        
        self.trigger_auswahl = widgets.Dropdown(
            options=[('Steigend', 'steigend'), ('Fallend', 'fallend')],
            value='steigend',
            description='Trigger-Flanke:',
            style={'description_width': 'initial'}
        )
        
        # Messsteuerung
        self.start_button = widgets.Button(
            description='Start',
            button_style='success',
            tooltip='Messung starten'
        )
        self.start_button.on_click(self.toggle_messung)
        
        self.einzelmessung_button = widgets.Button(
            description='Einzelmessung',
            button_style='info',
            tooltip='Einzelmessung durchführen'
        )
        self.einzelmessung_button.on_click(self.durchfuehre_einzelmessung)
        
        # Aufzeichnungssteuerung
        self.aufzeichnung_button = widgets.Button(
            description='Aufzeichnen ▶',
            button_style='warning',
            tooltip='Datenaufzeichnung starten/stoppen'
        )
        self.aufzeichnung_button.on_click(self.toggle_aufzeichnung)
        
        self.speichern_button = widgets.Button(
            description='Speichern 💾',
            tooltip='Aufgezeichnete Daten speichern'
        )
        self.speichern_button.on_click(self.speichere_daten)
        
        # Messparameterbox
        parameter_box = widgets.VBox([
            self.abtastrate_slider,
            self.zeitbasis_slider,
            widgets.HBox([self.trigger_checkbox, self.trigger_auswahl]),
            self.trigger_level_slider
        ])
        
        # Steuerungsbox
        steuerung_box = widgets.HBox([
            self.start_button, 
            self.einzelmessung_button,
            self.aufzeichnung_button,
            self.speichern_button
        ])
        
        # Messwertanzeige
        self.frequenz_text = widgets.HTML(value="<b>Frequenz:</b> -- Hz")
        self.amplitude_text = widgets.HTML(value="<b>Amplitude:</b> -- V")
        self.mittelwert_text = widgets.HTML(value="<b>Mittelwert:</b> -- V")
        self.rms_text = widgets.HTML(value="<b>RMS-Wert:</b> -- V")
        self.signaltyp_text = widgets.HTML(value="<b>Signaltyp:</b> --")
        
        messwert_box = widgets.HBox([
            widgets.VBox([self.frequenz_text, self.amplitude_text]),
            widgets.VBox([self.mittelwert_text, self.rms_text, self.signaltyp_text])
        ])
        
        # Echtzeit-Tab Layout
        self.echtzeit_tab.children = [
            parameter_box,
            steuerung_box,
            messwert_box
        ]
        
        # Plot-Container
        self.output = widgets.Output()
        
        # Hauptlayout
        self.haupt_layout = widgets.VBox([
            self.status_text,
            self.modus_tabs,
            self.output
        ])
        
        # GUI anzeigen
        display(self.haupt_layout)
        
        # Plot initialisieren
        with self.output:
            self.fig, self.ax = plt.subplots(figsize=(10, 6))
            self.line, = self.ax.plot([], [], lw=2)
            self.trigger_line = self.ax.axhline(y=0, color='r', linestyle='--', alpha=0.5)
            self.ax.set_xlim(0, 0.1)
            self.ax.set_ylim(-MAX_SPANNUNG, MAX_SPANNUNG)
            self.ax.set_xlabel('Zeit (s)')
            self.ax.set_ylabel('Spannung (V)')
            self.ax.set_title('Signalmonitor')
            self.ax.grid(True)
            plt.tight_layout()
            plt.show()
    
    def aktualisiere_plot(self, frame):
        """
        Aktualisiert den Plot mit neuen Daten.
        """
        # Pufferdaten abrufen
        puffer_daten = self.signalleser.puffer_aktualisieren()
        
        # Zeitachse berechnen
        zeitbasis = self.zeitbasis_slider.value
        t = np.linspace(0, zeitbasis, PUFFER_GROESSE)
        
        # Trigger anwenden, wenn aktiviert
        if self.trigger_checkbox.value:
            trigger_level = self.trigger_level_slider.value
            trigger_flanke = self.trigger_auswahl.value
            
            # Trigger-Linie aktualisieren
            self.trigger_line.set_ydata(trigger_level)
            
            # Trigger-Indizes finden
            if trigger_flanke == 'steigend':
                trigger_indizes = np.where((puffer_daten[:-1] < trigger_level) & (puffer_daten[1:] >= trigger_level))[0]
            else:
                trigger_indizes = np.where((puffer_daten[:-1] > trigger_level) & (puffer_daten[1:] <= trigger_level))[0]
            
            # Wenn Trigger gefunden, Daten neu ausrichten
            if len(trigger_indizes) > 0:
                trigger_index = trigger_indizes[0]
                if trigger_index + 1 < PUFFER_GROESSE:
                    puffer_daten = np.roll(puffer_daten, -trigger_index - 1)
        
        # Plot aktualisieren
        self.line.set_data(t, puffer_daten)
        
        # Analyse durchführen
        frequenz = self.analyse.berechne_frequenz(puffer_daten, self.abtastrate_slider.value)
        amplitude = self.analyse.berechne_amplitude(puffer_daten)
        mittelwert = self.analyse.berechne_mittelwert(puffer_daten)
        rms = self.analyse.berechne_effektivwert(puffer_daten)
        signaltyp = self.analyse.erkenne_signaltyp(puffer_daten, self.abtastrate_slider.value)
        
        # Messwertanzeige aktualisieren
        self.frequenz_text.value = f"<b>Frequenz:</b> {frequenz:.2f} Hz"
        self.amplitude_text.value = f"<b>Amplitude:</b> {amplitude:.3f} V"
        self.mittelwert_text.value = f"<b>Mittelwert:</b> {mittelwert:.3f} V"
        self.rms_text.value = f"<b>RMS-Wert:</b> {rms:.3f} V"
        self.signaltyp_text.value = f"<b>Signaltyp:</b> {signaltyp}"
        
        # Wenn Aufzeichnung aktiv, Daten speichern
        if self.aufzeichnung_aktiv:
            zeit_delta = time.time() - self.aufzeichnungs_start
            self.gemessene_daten.append({
                'Zeit': zeit_delta,
                'Spannung': puffer_daten[-1],
                'Frequenz': frequenz,
                'Amplitude': amplitude,
                'Mittelwert': mittelwert,
                'RMS': rms
            })
            self.status_text.value = f"<b>Status:</b> Aufzeichnung läuft... ({len(self.gemessene_daten)} Datenpunkte)"
        
        return self.line, self.trigger_line
    
    def toggle_messung(self, b):
        """
        Startet oder stoppt die kontinuierliche Messung.
        """
        self.laufend = not self.laufend
        
        if self.laufend:
            self.start_button.description = 'Stop'
            self.start_button.button_style = 'danger'
            self.status_text.value = "<b>Status:</b> Messung läuft"
            
            # Animation starten
            self.animation = FuncAnimation(
                self.fig, self.aktualisiere_plot, interval=50, 
                blit=True, cache_frame_data=False)
            
            with self.output:
                clear_output(wait=True)
                plt.show()
        else:
            if self.animation is not None:
                self.animation.event_source.stop()
                self.animation = None
            
            self.start_button.description = 'Start'
            self.start_button.button_style = 'success'
            self.status_text.value = "<b>Status:</b> Messung gestoppt"
    
    def durchfuehre_einzelmessung(self, b):
        """
        Führt eine Einzelmessung durch.
        """
        self.status_text.value = "<b>Status:</b> Einzelmessung wird durchgeführt..."
        
        # Animation stoppen, falls sie läuft
        if self.laufend:
            self.toggle_messung(None)
        
        # Anzahl der zu messenden Punkte berechnen
        anzahl_punkte = int(self.zeitbasis_slider.value * self.abtastrate_slider.value)
        
        # Messung durchführen
        t, spannungen, tats_abtastrate = self.signalleser.lese_block(anzahl_punkte)
        
        # Plot aktualisieren
        with self.output:
            clear_output(wait=True)
            
            self.fig, self.ax = plt.subplots(figsize=(10, 6))
            self.ax.plot(t, spannungen, lw=2)
            self.ax.set_xlim(0, self.zeitbasis_slider.value)
            self.ax.set_ylim(-MAX_SPANNUNG, MAX_SPANNUNG)
            self.ax.set_xlabel('Zeit (s)')
            self.ax.set_ylabel('Spannung (V)')
            self.ax.set_title(f'Einzelmessung (Abtastrate: {tats_abtastrate:.0f} Hz)')
            self.ax.grid(True)
            
            plt.tight_layout()
            plt.show()
        
        # Analyse durchführen
        frequenz = self.analyse.berechne_frequenz(spannungen, tats_abtastrate)
        amplitude = self.analyse.berechne_amplitude(spannungen)
        mittelwert = self.analyse.berechne_mittelwert(spannungen)
        rms = self.analyse.berechne_effektivwert(spannungen)
        signaltyp = self.analyse.erkenne_signaltyp(spannungen, tats_abtastrate)
        
        # Messwertanzeige aktualisieren
        self.frequenz_text.value = f"<b>Frequenz:</b> {frequenz:.2f} Hz"
        self.amplitude_text.value = f"<b>Amplitude:</b> {amplitude:.3f} V"
        self.mittelwert_text.value = f"<b>Mittelwert:</b> {mittelwert:.3f} V"
        self.rms_text.value = f"<b>RMS-Wert:</b> {rms:.3f} V"
        self.signaltyp_text.value = f"<b>Signaltyp:</b> {signaltyp}"
        
        self.status_text.value = "<b>Status:</b> Einzelmessung abgeschlossen"
    
    def toggle_aufzeichnung(self, b):
        """
        Startet oder stoppt die Datenaufzeichnung.
        """
        self.aufzeichnung_aktiv = not self.aufzeichnung_aktiv
        
        if self.aufzeichnung_aktiv:
            # Neue Aufzeichnung starten
            self.gemessene_daten = []
            self.aufzeichnungs_start = time.time()
            self.aufzeichnung_button.description = 'Aufzeichnung ⏹'
            self.aufzeichnung_button.button_style = 'danger'
            self.status_text.value = "<b>Status:</b> Aufzeichnung gestartet"
            
            # Sicherstellen, dass die Messung läuft
            if not self.laufend:
                self.toggle_messung(None)
        else:
            self.aufzeichnung_button.description = 'Aufzeichnen ▶'
            self.aufzeichnung_button.button_style = 'warning'
            self.status_text.value = f"<b>Status:</b> Aufzeichnung gestoppt ({len(self.gemessene_daten)} Datenpunkte)"
    
    def speichere_daten(self, b):
        """
        Speichert die aufgezeichneten Daten in eine CSV-Datei.
        """
        if not self.gemessene_daten:
            self.status_text.value = "<b>Status:</b> Keine Daten zum Speichern vorhanden"
            return
        
        # DataFrame erstellen
        df = pd.DataFrame(self.gemessene_daten)
        
        # Dateiname mit Zeitstempel erstellen
        zeitstempel = datetime.now().strftime("%Y%m%d_%H%M%S")
        dateiname = f"signalmonitor_daten_{zeitstempel}.csv"
        
        # Daten speichern
        df.to_csv(dateiname, index=False)
        
        self.status_text.value = f"<b>Status:</b> Daten gespeichert als '{dateiname}'"

# Signalmonitor starten
signalmonitor = SignalmonitorGUI()